### Objective: To test pipeline end to end and write additional code as needed to link them together. 

In [17]:
import pandas as pd
from pathlib import Path
import nglview as ngl
import numpy as np

from developability.energy_minimization import EnergyMinimizer
from developability.electrostatics import APBS
from developability.surface import SurfacePotential
from developability.descriptors import descriptor_pipeline, map3to1
from developability.utils import renumber_pdb, extract_sequence_from_pdb, ls, clean_logs

#### Set up path to antibodies. 
These are therapeutic antibodies. 

In [18]:
pdbs_path = Path('/Users/ChristopherRivera/Desktop/pdbs')

In [19]:
ab = Path('/Users/ChristopherRivera/Desktop/') /'fontolizumab.pdb'
# renumber the sequence to be sure. 
renumber_pdb(ab)
ab = Path('/Users/ChristopherRivera/Desktop/') /'fontolizumab.renumbered.pdb'
ngl.show_structure_file(str(ab))

NGLWidget()

In [20]:
def run_processing_pipeline(input_pdb, nanoshaper_options=None):
    """ Runs processing pipeline
    Args: 
        input_pdb (_type_): _description_
        nanoshaper_options(dict)
    Returns
    """
    
    # minimize the energy 
    minimizer=EnergyMinimizer(input_pdb)
    minimized_pdb = minimizer.minimize_energy()

    # compute electrostatics
    apbs = APBS(minimized_pdb )
    pqr_file, dx_file = apbs.calculate_potential()

    # compute the surface
    surface = SurfacePotential(pqr_file, dx_file, nanoshaper_options=None)
    residue_pot_file = surface.calculate_surface_potential()

    # get the sequence from the input pdb
    sequences = extract_sequence_from_pdb(input_pdb)
    light_chain_seq = sequences['L']
    heavy_chain_seq = sequences['H']

    # Calculate the descriptors
    antibody_name = input_pdb.name.split('.')[0]
    descriptors = descriptor_pipeline(light_chain_seq, heavy_chain_seq,residue_pot_file, antibody_name)
    
    # clean out logs in execution location
    clean_logs()
    
    return descriptors

In [21]:
descriptors = run_processing_pipeline(ab)

Starting pdb4amber with /Users/ChristopherRivera/Desktop/fontolizumab.renumbered.pdb. 

2023-10-09 11:55:53,691 [MainThread  ] [INFO ]  Executing biobb_amber.pdb4amber.pdb4amber_run Version: 4.1.0
2023-10-09 11:55:53,692 [MainThread  ] [INFO ]  Copy: /Users/ChristopherRivera/Desktop/fontolizumab.renumbered.pdb to /Users/ChristopherRivera/Projects/developability/Notebooks/bb4b8732-67b0-457e-8609-4712e1e125bc
2023-10-09 11:55:53,694 [MainThread  ] [INFO ]  Creating 83dc423d-7db3-4296-a84f-0ebf861419a0 temporary folder
2023-10-09 11:55:53,695 [MainThread  ] [INFO ]  pdb4amber -i /Users/ChristopherRivera/Projects/developability/Notebooks/bb4b8732-67b0-457e-8609-4712e1e125bc/fontolizumab.renumbered.pdb -o /Users/ChristopherRivera/Projects/developability/Notebooks/bb4b8732-67b0-457e-8609-4712e1e125bc/pdb4amber.pdb -y 

2023-10-09 11:55:54,811 [MainThread  ] [INFO ]  Exit code 0

2023-10-09 11:55:54,814 [MainThread  ] [INFO ]  
Summary of pdb4amber for: /Users/ChristopherRivera/Projects/devel

TypeError: 'str' object is not callable

In [ ]:
np.round(descriptors.T,2)